### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2024-11-27 12:30:06--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.80.18, 2620:100:6021:18::a27d:4112
Подключение к www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [переход]
--2024-11-27 12:30:08--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc305df56ddda159494d27e31140.dl.dropboxusercontent.com/cd/0/inline/CfLPFAFwLG_LzxMQAffGbh6XgJwPejoxMt5kTH9rpEyljSpm2xoLZpuXo57DsIwD7AZv_8uNBhy8O6G0JufXojdl0WqmK4XcLYuSYJdQqXvPoZKZSlfrlqVOlLczxQ0UlqQ/file?dl=1# [переход]
--2024-11-27 12:30:09--  https://uc305df56ddda159494d27e31140.

,author,day,id,link,month,summary,tag,title,year
11275,"[{'name': 'Alain Rakotomamonjy'}, {'name': 'Re...",2,1507.00438v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,We introduce a novel algorithm for solving lea...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",DC Proximal Newton for Non-Convex Optimization...,2015
33301,[{'name': 'Raphaël Cerf'}],19,1005.3390v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Based on speculations coming from statistical ...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Critical control of a genetic algorithm,2010
37867,"[{'name': 'Kristi Kuljus'}, {'name': 'Jüri Lem...",18,1002.3509v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We consider the maximum likelihood (Viterbi) a...,"[{'term': 'math.PR', 'scheme': 'http://arxiv.o...",Asymptotic risks of Viterbi segmentation,2010
28794,"[{'name': 'Bruno Stuner'}, {'name': 'Clément C...",24,1707.07432v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Offline handwritten text line recognition is a...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",LV-ROVER: Lexicon Verified Recognizer Output V...,2017
11256,"[{'name': 'Colin H. LaMont'}, {'name': 'Paul A...",19,1506.05855v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,A central problem in statistics is model selec...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Information-based inference for singular model...,2015


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [9]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

from nltk.tokenize import WordPunctTokenizer

wp_tokenizer = WordPunctTokenizer()
lines = [" ".join(wp_tokenizer.tokenize(line)).lower() for line in lines]

In [10]:
sorted(lines, key=len)[:3]

['differential contrastive divergence ; this paper has been retracted .',
 'what does artificial life tell us about death ? ; short philosophical essay',
 'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .']

In [11]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [32]:
l = 'aaaaaaaa bbbbbb cccccccc ddddddd'
i = 1
n = 2
ctp = tuple()

if i < n:
    ctp = ('UNK',) * (n - i)
    ctp += tuple(tok for tok in l.split()[:i])
    # for tok in l.split()[:i]:
    #     ctp += (tok,)

ctp


('UNK', 'aaaaaaaa')

In [101]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        tokens = line.split() + [EOS]
        padding_tokens = [UNK] * (n-1) + tokens

        for i in range(len(tokens)):
            prefix = tuple(padding_tokens[i:i + n - 1])
            word = tokens[i]

            counts[prefix][word] += 1
    
    
    return counts


In [102]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [75]:
a = ("bbbbb", 'vvvvv')
" ".join(a)
type(a)

tuple

In [84]:
for prefix, count in dummy_counts.items():
    print(prefix)
    print(count)
    print(type(prefix), type(count))
    break

('_UNK_', '_UNK_')
Counter({'a': 278, 'p': 153, 'the': 127, 'in': 97, 'an': 76, 'on': 47, 'network': 18, 'learning': 13, 'are': 10, 'some': 6, 'deep': 5, 'recognition': 5, 'using': 5, 'complex': 4, 'introduction': 4, 'sat': 4, 'decision': 3, 'text': 3, 'entropy': 3, 'how': 3, 'time': 2, 'about': 2, 'convex': 2, 'calculate': 2, 'group': 2, 'cornell': 2, 'proceedings': 2, 'creating': 1, 'defeasible': 1, 'essence': 1, 'mining': 1, 'icon': 1, 'glottochronologic': 1, 'utility': 1, 'activitynet': 1, 'comments': 1, 'automatic': 1, 'resource': 1, 'advances': 1, 'quantified': 1, 'beyond': 1, 'norm': 1, 'unary': 1, 'why': 1, 'philosophy': 1, 'word': 1, 'attack': 1, 'defensive': 1, 'technical': 1, 'approximated': 1, 'solving': 1, 'agent': 1, 'differential': 0, 'what': 0, 'computational': 0, 'weak': 0, 'statistical': 0, 'serious': 0, 'preprocessing': 0, 'liquid': 0, 'towards': 0, 'temporized': 0, 'backpropagation': 0, 'random': 0, 'glottochronology': 0, 'convolutional': 0, 'fitness': 0, 'flip': 0,

In [81]:
isinstance(" ".join(list(dummy_counts.keys())[0]), str)

True

In [ ]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        for prefix, word_dict in counts.items():
            count_sum = sum(word_dict.values())
            for word, count in word_dict.items():
                self.probs[prefix][word] = (count) / (count_sum)
        # populate self.probs with actual probabilities
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [104]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

In [105]:
p_initial = dummy_lm.get_possible_next_tokens('')
p_initial['learning']


0.02

In [109]:
p_initial = dummy_lm.get_possible_next_tokens('')
list(p_initial)

['differential',
 'what',
 'p',
 'computational',
 'weak',
 'creating',
 'defeasible',
 'essence',
 'deep',
 'statistical',
 'complex',
 'serious',
 'preprocessing',
 'liquid',
 'mining',
 'towards',
 'a',
 'icon',
 'recognition',
 'glottochronologic',
 'the',
 'utility',
 'temporized',
 'backpropagation',
 'random',
 'network',
 'glottochronology',
 'using',
 'time',
 'convolutional',
 'fitness',
 'flip',
 'autonomous',
 'activitynet',
 'decision',
 'text',
 'discrimination',
 'are',
 'extraction',
 'comments',
 'learning',
 'automatic',
 'resource',
 'advances',
 'exploration',
 'quantified',
 'in',
 'introduction',
 'beyond',
 'norm',
 'about',
 'unary',
 'some',
 'convex',
 'why',
 'neurocontrol',
 'on',
 'philosophy',
 'parallels',
 'an',
 'calculate',
 'group',
 'entropy',
 'word',
 'guarded',
 'cornell',
 'semistability',
 'proceedings',
 'attack',
 'standardization',
 'defensive',
 'piecewise',
 'how',
 'technical',
 'sat',
 'approximated',
 'solving',
 'agent']

In [106]:


p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [107]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [124]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs_next_tokens = lm.get_possible_next_tokens(prefix)
    if not probs_next_tokens:
        return None
    
    probs = list(probs_next_tokens.values())
    next_tokens = list(probs_next_tokens.keys())

    if temperature == 0.0:
        max_prob = max(probs)
        idxs_next_tokens = [i for i in range(len(probs)) if probs[i] == max_prob]
        idx_next_token = np.random.choice(idxs_next_tokens, size=1)[0]
        return next_tokens[idx_next_token]

    probs = [prob ** (1 / temperature) for prob in probs]
    total_sum_probs = sum(probs)
    normalized_probs = [prob / total_sum_probs for prob in probs]

    idx_next_token = np.random.choice(range(len(next_tokens)), p=normalized_probs,size=1)[0]
    
    return next_tokens[idx_next_token]



In [118]:
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
test_freqs['not']

381

In [125]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [134]:
prefix = 'number' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

number game ; clarin ( common requirements with a given text . we argue that this approach , our novel method for improving the comparison at this result by replacing the initial data , on all three sentiment and tweet level . similar to that actor based on the classification accuracy of 52 . 80 \%$ on the partially observed in time - resolutions . we evaluate our approach has the best of our approach on several classic data augmentation . finally , we tried didn ' t know answers are given a single arm policies is that they improve upon


In [135]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two - player games , and an associated set of candidate solutions are found to be mapped to a target hypothesis that the proposed method . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [165]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """

    total_token_counts = 0
    total_log_probs_sum = 0
    for line in lines:
        tokens = line.split() + [EOS]
        log_sum = 0
        for i in range(len(tokens)):
            prefix = " ".join(tokens[:i])
            word = tokens[i]
            log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))

        total_log_probs_sum += log_sum
        total_token_counts += len(tokens)

    avg_log_probs = total_log_probs_sum/total_token_counts
    perplexity_probs = np.exp(-avg_log_probs)
    print(f"total_token_counts = {total_token_counts}")
            
    return perplexity_probs

In [166]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

total_token_counts = 2877
total_token_counts = 2877
total_token_counts = 2877
total_token_counts = 9
Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/tmp/ipykernel_84659/2076743267.py:20: RuntimeWarning: divide by zero encountered in log
  log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [150]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/tmp/ipykernel_84659/4037025384.py:21: RuntimeWarning: divide by zero encountered in log
  log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28543
N = 3, Perplexity = 61999196239911532363776.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [126]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [127]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [167]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

total_token_counts = 1951241
N = 1, Perplexity = 977.67559
total_token_counts = 1951241
N = 2, Perplexity = 470.48021
total_token_counts = 1951241
N = 3, Perplexity = 3679.44765


In [168]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the trips mechanism names lowest morality employer tries conversational lineages placeraider blinded dowty onenet distributedly pronet dialectic aapm mpmp illu refered confirm %-- dvas minded structions decoding uberon gröbner mosquito valpola tbptt cooler underdiagnosis pcns bu4d winnowed concretely participatory uhhuh fires dom se owned schemein psxasj tabucol pancreatic aus zeevi maoyi corporal account sysem battista aswdr 6704 affordance nearables ofms gesammelt volleys iconip interfering coaching adl urdu informant dlan spacey buntine lennard nilm refuted au15 amodal knet prosodi inborn prioritize unifilar virgin hnp3 poster neurotransmitter initialising ccar framenet comparably seems sine 145 p2p roadside wrapper glsr 8s tnr deflecting =$ mgan


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [175]:
a = [1,1,1,12,2,3]
a[-5:]

[1, 1, 12, 2, 3]

In [177]:
fff = {'t' : 1, 'p': 2}

for ff, f in fff.items():
    fff[ff] += f

fff

{'t': 2, 'p': 4}

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        super().__init__(lines, n)

        self.delta = delta
        self.unigrams = self._compute_unigrams()
    
    def _compute_unigrams(self):
        total_count_words = sum(sum(word_counter.values()) for word_counter in  self.probs.values())

        unigram = {}

        for word_counter in self.probs.values():
            for word, count in word_counter.items():
                if word in unigram:
                    unigram[word] += count
                else:
                    unigram[word] = count   

        unigram = {word: count/total_count_words for word, count in unigram.items()}   

        return unigram

    def _compute_lambda(self, prefix):

        all_postfixes = self.probs[prefix]
        total_count_non_zero = len(all_postfixes)
        total_count_word_after_prefix = sum(all_postfixes.values())

        _lambda = self.delta * (total_count_non_zero) / (total_count_word_after_prefix)

        return _lambda

    def get_possible_next_tokens(self, prefix):
        next_token_probs_kn = {word : 0 for word, _ in self.probs[prefix]}

        tokens = prefix.split()
        n = len(tokens)
        for i in range(1, n + 1):
            if n == 1:
                curr_lambda = self._compute_lambda(tuple([UNK] * n))
                for word, _ in next_token_probs_kn.items():
                    next_token_probs_kn[word] += curr_lambda * self.unigrams[word]
            else:
                curr_prefix = [UNK] * (n - i) + tokens[-i:]
                prefix_tuple = tuple(curr_prefix)

                word_counts = {word : self.probs[prefix_tuple][word] for word in next_token_probs_kn}
                total_count_word_after_prefix = sum((self.probs[prefix_tuple].values()))
                curr_lambda = self._compute_lambda(prefix_tuple)

                for word in next_token_probs_kn:
                    next_token_probs_kn[word] += (max(0, word_counts[word] - self.delta)/total_count_word_after_prefix + curr_lambda * next_token_probs_kn[word])
            
        
        
    def get_next_token_prob(self, prefix, next_token):
        next_token_probs = self.get_possible_next_tokens(prefix)

        return next_token_probs.get(next_token, 0)

In [179]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

AttributeError: 'KneserNeyLanguageModel' object has no attribute 'vocab'

In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))